## <center> Introduction to integration methdos </center>

In [1]:
from scipy import integrate
import numpy as np
import matplotlib.pyplot as plt

### Univariate integration
The numerical quadrature routines in the SciPy integrate module can be categorized into two types: routines that take
the integrand as a Python function and routines that take arrays with samples of the integrand at given points. The
functions of the first type use Gaussian quadrature (quad, quadrature, fixed_quad), while functions of the second type
use Newton-Cotes methods (trapz, simps, and romb).

**Example** --> Gaussian quadrature

$$ y = \int_{-\infty}^{\infty} e^{-x^2}dx $$

*Note*:
The quad routine supports infinite integration limits. To represent integration limits that are infinite, we use the floating-point representation of infinity, float('inf')



In [2]:
fun = lambda x, a: np.exp(-x ** a)

val, err = integrate.quad(fun, -np.inf, np.inf, args=(2))  # args=() substitutes missing variables
print(val, err)

# The quad function returns a tuple that contains the numerical estimate of the integral, val, 
# and an estimate of the absolute error, err.

1.7724538509055159 1.4202636781830878e-08


The quad function is also able to handle many integrals with integrable singularities.

**Example** --> Non-continuous function

$$ y = \int_{-1}^{1} \frac{1}{\sqrt{|x|}}dx $$

In [6]:
f = lambda x: 1 / np.sqrt(abs(x))

# value, error = integrate.quad(f, -1, 1)
# print(value) --> (inf, inf)

value, error = integrate.quad(f, -1, 1, points=[0])  # points=[] states the points not to be evaluated
print(value)

3.999999999999999


### Tabulated  Integrand
We have seen that the quad routine is suitable for evaluating integrals when the integrand is specified using a Python function that the routine can evaluate at arbitrary points (which is determined by the specific quadrature rule). However, in many situations we may have an integrand that is only specified at predetermined points, such as evenly spaced points in the integration interval [a, b]. This type of situation can occur, for example, when the integrand is obtained from experiments or observations that cannot be controlled by the particular integration routine. In this case we can use a Newton-Cotes quadrature, such as the midpoint rule, trapezoid rule, or Simpson’s rule.

**Example**
$$ y = \int_{0}^{2} \sqrt{x}dx $$


In [9]:
fun_sqrt = lambda x: np.sqrt(x)
a, b = 0, 2

points = np.arange(10, 110, 10)
for point in points:
    x = np.linspace(a, b, point) # More points, gives a better accuracy
    y = fun_sqrt(x)
    # Trapezoidal rule
    val_trapz = integrate.trapz(y, x)
    print(val_trapz)

1.8652953655957172
1.8788448242701692
1.8819931375475458
1.883281362131247
1.883952907509645
1.8843544824312572
1.8846169553415666
1.8847995720659774
1.8849326589234765
1.8850331850950104


In [10]:
# Simpson rule
val_simps = integrate.simps(y, x)
print(val_simps)

1.885260412687424


The Romberg method is a Newton-Cotes method but one that uses Richardson extrapolation to accelerate the convergence
of the trapezoid method; however this method does require that the sample points are evenly spaced and also that
there are $2^n+1$ sample points, where n is an integer. Like the trapz and simps methods, romb takes an array with
integrand samples as first argument, but the second argument must (if given) be the sample-point spacing $dx$.


In [11]:
pow = 1 + 2 ** 6
x_romb = np.linspace(a, b, pow)
dx = x_romb[1] - x_romb[0]
y_romb = fun_sqrt(x_romb)
val_romb = integrate.romb(y_romb, dx=dx)
print(val_romb)

1.8852392847412136


### Multiple integration
Multiple integrals, such as double integrals and triple integrals can be evaluated using the dblquad and tplquad
functions from the SciPy integrate module. Also, integration over n variables over some domain D, can be evaluated
using the nquad function. These functions are wrappers around the single-variable quadrature function quad, which is
called repeatedly along each dimension of the integral.

The double integral routine dblquad can evaluate integrals on the form:
$$ y = \int_{a}^{b} \int_{h(x)}^{g(x)} f(x,y)dxdy $$

and it has the function signature dblquad(f, a, b, g, h), where f is a Python function for the integrand, a and b are constant integration limits along the x dimension, and gand f are Python functions (taking x as argument) that specify the integration limits along the y dimension. 


The tplquad function can compute integrals on the form:

$$ y = \int_{a}^{b} \int_{h(x)}^{g(x)} \int_{r(x,y)}^{q(x,y)} f(x,y,z)dxdydz $$

which is a generalization of the double integral expression computed with dblquad. It additionally takes two Python functions as arguments, which specifies the integration limits along the z dimension. These functions take two arguments, x and y, but note that g and h still only take one argument (x).


In [12]:
# Example for triple integral (double, lambda fun oly takes x dependant variables in the first integral)

def f(x, y, z):
    return np.exp(-x ** 2 - y ** 2 - z ** 2)


g, h = 0.1, 1
i, j = lambda x: 2 * x ** 2 + 15 * x + 3, lambda x: np.sqrt(x)
k, l = lambda x, y: x*y**2 - x + 1/y, lambda x, y: np.sqrt(1/(x*y))

res_triple = integrate.tplquad(f, g, h, i, j, k, l)
print(res_triple)

(-0.011472621424148842, 1.4480124059831105e-08)
